In [ ]:
DATE = "0928_resnet152"
DATA_TO_TEST = 'all' # 'abn1' or 'abn2' or 'normal'
MODEL_EPOCH = '_100ep' ###### 아래 스냅샷 파일과 일치하는지 반드시 확인!!!!!!! ######

SCORE_VALUE = 0.1
PRINT_BENIGN_DETECTION = False

import os
import pandas as pd

result_save_base_path = 'D:/Work/NCC/han_test/results/'
result_save_path = os.path.join(result_save_base_path, DATE, DATA_TO_TEST + MODEL_EPOCH + '_' + str(SCORE_VALUE))
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
model = keras.models.load_model('snapshots/0928_resnet50_ep100_with_valid/resnet50_csv_20.h5', custom_objects=custom_objects)
#model = keras.models.load_model('snapshots/0928_resnet102_ep100_with_valid/resnet102_csv_20.h5', custom_objects=custom_objects)
#model = keras.models.load_model('snapshots/0927_resnet152_ep100_with_valid/resnet152_csv_07.h5', custom_objects=custom_objects)

## Initialize data generators

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn1.csv'
elif DATA_TO_TEST == 'abn2':
    csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn2.csv'
elif DATA_TO_TEST == 'normal':
    csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_normal.csv'
elif DATA_TO_TEST == 'all':
    #csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_anno_for_win.csv'
    csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_for_win.csv'
else:
    print('WRONG DATA CSV PATH!')
    raise

# create a generator for testing data
val_generator = CSVGenerator(
    csv_path,
    #'D:/Work/NCC/data/new_trainset/class_2_malig_or_norm.csv',
    'D:/Work/NCC/data/new_trainset/class_malig_and_norm.csv',
    transform_generator=val_image_data_generator,
    batch_size=1
)

test_file_df = pd.read_csv(csv_path, header=None)
num_of_test_files = len(test_file_df)
print(num_of_test_files)

test_file_list = list(test_file_df[0]) #csv에서 읽어온 테스트 이미지파일 정보 목록
test_file_list.sort(key=lambda x: '-'.join(x.split('-')[:2])) #테스트 파일 목록을 groupby 하기 위해서 정렬

## Run detection

In [ ]:
image_with_no_detection = []
image_with_more_than_one_anno = []
num_of_anno = 0
correct_detection = 0

In [ ]:
img_index_list = list(range(0, num_of_test_files))
img_index_list_temp = img_index_list.copy()
print(len(img_index_list))

#테스트 파일목록 Groupping
groupby_key_list = [] #그룹명(테스트 파일 단위 그룹 명) 목록
groupby_file_list = [] #그룹화된 테스트파일 목록
groupby_detected_number_for_files = [] # 검출(detect)된 수에 대한 그룹화된 목록(그룹화된 테스트 목록 구조와 동일하게 매핑)
#테스트 파일 목록을 group by하여 list에 저장
#groupby key(그룹핑 기준(단위)는 파일명을 '-'로 split 한뒤 앞쪽 2번째 요소까지 추출함)
for key, group in groupby(test_file_list, key=lambda x: '-'.join(x.split('-')[:2])):
    #그룹 명(key) 목록
    groupby_key_list.append(key) 
    temp = list(group)
    #그룹화된 전체 테스트파일 목록 
    groupby_file_list.append(temp) 
    #그룹화 된 테스트 파일별 검출정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    groupby_detected_number_for_files.append([0]*len(temp))

In [ ]:
number_of_not_detected_abnormal_case = 0
number_abnormal_detect_from_normal = 0
image_path_list = []
M_max_scores = []
N_max_scores = []
cur_set = ''
number_of_detected_case_per_set = 0 #1 set당 detect된 이미지 case 수
number_of_case_for_not_detected_set = 0 #아무것도 detect 되지 않은 set에 속한 전체 case 수
number_of_not_detected_set = 0 #아무것도 detect되지 않은 set 수
b_is_cur_set = True
number_of_case_per_set = 0 #1 set당 이미지 case 수



for index in img_index_list:
    pass_this_file = False
    
    # load image
    image, image_path = val_generator.load_image(index, get_image_path=True)
    #print('image :::::::::: {}'.format(image_path))
    image_name_split = image_path.split('/')[-1].split('-')
    

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    #annotations = val_generator.load_annotations(index)
    

    # process image
#     start = time.time()
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
#     print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

    # correct for image scale
    detections[0, :, :4] /= scale
    
    '''anno_center = []
    
    if len(annotations) > 2:
        print("MANY ANNO!!! len: " +str(len(annotations)) + ' path: '+ image_path )
        image_with_more_than_one_anno.append(image_path)'''
    
    # visualize annotations
    '''for annotation in annotations:
        num_of_anno += 1
        
        label = int(annotation[4])
        b = annotation[:4].astype(int)
        
        anno_center.append((b[0]+b[2])/2)
        anno_center.append((b[1]+b[3])/2)
        
        
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (50, 50, 255), 2)
        caption = "{}".format(val_generator.label_to_name(label))
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)'''
            

    # 현 이미지에서 추출된 anchor들 중에 주어진 detection 조건(score 및 기타)을 만족하는 anchor 수
    number_of_proper_detection = 0

    image_path_list.append(image_path)
    
    if index == 0:
        cur_set = image_name_split[0]+'-'+image_name_split[1]
        #print("cur_set from index=0::::::::::::::::::: {}".format(cur_set))
        number_of_case_per_set = 1
    else:
        if cur_set not in image_path.split('/')[-1]:
            #print("image_path.split('/')[-1]::::::::::::{}".format(image_path.split('/')[-1]))
            cur_set = image_name_split[0]+'-'+image_name_split[1]
            #print("cur_set ::::::::::::::::::: {}".format(cur_set))
            b_is_cur_set = False
        else:
            #b_is_cur_set = True
            number_of_case_per_set += 1
        
    M_max_score = 0
    N_max_score = 0
    
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if label == 0:
            if M_max_score < score:
                M_max_score = score    
        else:
            if N_max_score < score:
                N_max_score = score
        
        if score < SCORE_VALUE or label != 0:
            continue

        number_of_proper_detection += 1
        
        b = detections[0, idx, :4].astype(int)
            
        caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)
        
        
        ######### BENIGN 출력 여부 설정 #########
        #if PRINT_BENIGN_DETECTION is False:
        #    if 'B' in caption:
        #        continue
                
                
        '''if pass_this_file is False and len(anno_center) == 2 and b[0]<anno_center[0]<b[2] and b[1]<anno_center[1]<b[3]:
            correct_detection += 1
            pass_this_file = True
            
            # remove the detected image from the list
            img_index_list_temp.remove(index)'''
       
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
            
    '''if number_of_proper_detection == 0:
        image_with_no_detection.append(image_path)'''
    
    M_max_scores.append(M_max_score)
    N_max_scores.append(N_max_score)
    
    if not b_is_cur_set: #이미지 set가 변경되었을 경우
        if number_of_detected_case_per_set < 1: #이전 set에 속한 이미지 case에서 detect된 것이 없을 경우
            number_of_case_for_not_detected_set += number_of_case_per_set
            number_of_not_detected_set += 1
        number_of_detected_case_per_set = 0
        b_is_cur_set = True
        number_of_case_per_set = 1    
    

    #이미지에서 (조건에 부합되게)detect된 것이 있을 경우
    if number_of_proper_detection > 0:
        if 'normal' in image_path:
            number_abnormal_detect_from_normal += 1            
        #if b_is_cur_set: #현재 이미지가 같은 set에 속한 경우...
            number_of_detected_case_per_set += 1
    else: #detect된 것이 없을 경우
        if 'abn' in image_path:
            #print("missing image ::::::::::::::: {}".format(image_path))
            #print("missing score ::::::: M {:.3f},N {:.3f}".format(M_max_score,N_max_score))
            number_of_not_detected_abnormal_case += 1
    
    #cv2.imwrite(os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(index)+'.jpg'), draw)

print('Number of abnormal case not detected :::::::::: {}'.format(number_of_not_detected_abnormal_case))
print('Number of abnormal case set not detected::::::: {}({} images)'.format(number_of_not_detected_set, number_of_case_for_not_detected_set))
print('Number of abnormal case detected from normal::: {}'.format(number_abnormal_detect_from_normal))    


## Create CSV

In [ ]:
result_data_df = pd.DataFrame({'img_path':image_path_list, 'abnormal_max_score':M_max_scores, 'normal_max_score':N_max_scores})
result_data_df = result_data_df[['img_path', 'abnormal_max_score', 'normal_max_score']]

In [ ]:
result_data_df.to_csv('D:/Work/NCC/han_test/results/test_result.csv', header=False, index=False)

In [ ]:
print("DONE!" + DATA_TO_TEST + MODEL_EPOCH)

## Check remained images

In [ ]:
# check remained images
img_index_list = img_index_list_temp.copy()

print("FAILED TO DETECT:")

detection_failed = []

for img_index in img_index_list:
    image, image_path = val_generator.load_image(img_index, get_image_path=True)
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    annotations = val_generator.load_annotations(img_index)
    
    if len(annotations) != 0: # images that the model failed to detect
        detection_failed.append(image_path)
        
detection_failed.sort()

for path in detection_failed:
    print(path)

In [ ]:
print(correct_detection, num_of_anno)
if len(image_with_more_than_one_anno) != 0:
    print("문제있다!!" + str(image_with_more_than_one_anno))

In [ ]:
case_with_no_detection = []

test_file_path_list = test_file_df[0].tolist()

image_with_no_detection.sort()
for l in image_with_no_detection:        
    splitted = l.split('-')[:2]
    case_path = '-'.join(splitted)
    
    if case_path in case_with_no_detection:
        continue
    
    same_case = [k for k in image_with_no_detection if case_path in k]
    if len(same_case) == len([k for k in test_file_path_list if case_path in k]):
        case_with_no_detection.append(case_path)

In [ ]:
print(len(case_with_no_detection))

for l in case_with_no_detection:
    print(l)